In [20]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [8]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 7.2 MB/s eta 0:00:00


In [18]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 8.6 MB/s eta 0:00:00


In [19]:
import warnings
warnings.filterwarnings("ignore")

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import GPT2Tokenizer , GPT2LMHeadModel , Trainer , TrainingArguments
from datasets import Dataset
from transformers import DataCollatorForLanguageModeling

In [21]:
df = pd.read_csv(r"/content/Mental_Health_FAQ.csv")
df.head()          # show first 5 rows

,Question_ID,Questions,Answers
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,6361820,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."


In [22]:
# get summary data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Question_ID  98 non-null     int64 
 1   Questions    98 non-null     object
 2   Answers      98 non-null     object
dtypes: int64(1), object(2)
memory usage: 2.4+ KB


In [23]:
df.columns

Index(['Question_ID', 'Questions', 'Answers'], dtype='object')

- Question_ID : unique idenitifier
- Questions : Questions like users want to ask about
- Answers : Answers of Questions

In [24]:
# drop [ID
df.drop(columns = ["Question_ID"] , axis =1, inplace=True)
df

,Questions,Answers
0,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,Can people with mental illness recover?,"When healing from mental illness, early identi..."
...,...,...
93,How do I know if I'm drinking too much?,Sorting out if you are drinking too much can b...
94,"If cannabis is dangerous, why are we legalizin...","Cannabis smoke, for example, contains cancer-c..."
95,How can I convince my kids not to use drugs?,You can't. But you can influence their capacit...
96,What is the legal status (and evidence) of CBD...,Cannabidiol or CBD is a naturally occurring co...


In [25]:
df.shape

(98, 2)

In [26]:
# check duplictaes
df[df.duplicated()]

,Questions,Answers


In [27]:
#  apply the answer to its question  ( GPT expect this format ->  Q: .....\nA:.......)

df["text"] = df.apply(lambda row: f"Q: {row['Questions']}\nA: {row['Answers']}", axis=1)

In [28]:
df["text"]

,text
0,Q: What does it mean to have a mental illness?...
1,Q: Who does mental illness affect?\nA: It is e...
2,Q: What causes mental illness?\nA: It is estim...
3,Q: What are some of the warning signs of menta...
4,Q: Can people with mental illness recover?\nA:...
...,...
93,Q: How do I know if I'm drinking too much?\nA:...
94,"Q: If cannabis is dangerous, why are we legali..."
95,Q: How can I convince my kids not to use drugs...
96,Q: What is the legal status (and evidence) of ...


In [87]:
df.to_csv(r"/content/processed_Mental_Health_FAQ.csv")

In [29]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")


# GPT-2 tokenizer doesn't have a dedicated padding token, which is needed for batch processing.
tokenizer.add_special_tokens({"pad_token": "[PAD]"})

# Resize the model's embedding layer to include the new token's vector.
model.resize_token_embeddings(len(tokenizer))

# Inform the model's configuration of the padding token's ID.
model.config.pad_token_id = tokenizer.pad_token_id

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [30]:
# convert Datafram to Dataset type (since pre-trained models like GPT expect Dataset Formate)

dataset = Dataset.from_pandas(df[["text"]])
dataset

Dataset({
    features: ['text'],
    num_rows: 98
})

In [31]:
# define Tokenization Function to get input_ids & attention mask


def tokenize_function(examples):
    return tokenizer(examples["text"],padding="max_length",
                             truncation=True,max_length=128  )

tokenized_dataset = dataset.map(tokenize_function, batched=True)



Map:   0%|          | 0/98 [00:00<?, ? examples/s]

In [32]:
tokenized_dataset

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 98
})

In [33]:
# split Dataset to train ,test
train_test = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test["train"]
test_dataset = train_test["test"]

In [34]:
# GPT is CLM (Casual Langauge Model) : predict next token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm=False)

In [35]:
#  training args

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=6,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.1,
    logging_dir="./logs",
    logging_steps=10,
    logging_first_step=True,
    logging_strategy="steps",
    report_to="none",
    save_steps=300,
    overwrite_output_dir=True,
    eval_strategy="epoch",
    disable_tqdm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)
trainer.train()


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50257}.
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.301900,2.625068
2,2.776600,2.549840
3,2.776600,2.514910
4,2.411300,2.496649
5,2.411300,2.485431
6,2.268100,2.480432


TrainOutput(global_step=30, training_loss=2.502848672866821, metrics={'train_runtime': 1138.4756, 'train_samples_per_second': 0.411, 'train_steps_per_second': 0.026, 'total_flos': 30571167744000.0, 'train_loss': 2.502848672866821, 'epoch': 6.0})

In [36]:
trainer.evaluate()

{'eval_loss': 2.4804320335388184,
 'eval_runtime': 11.8297,
 'eval_samples_per_second': 1.691,
 'eval_steps_per_second': 0.169,
 'epoch': 6.0}

# Testing

In [37]:
trainer.save_model("./mental_health_gpt2")
tokenizer.save_pretrained("./mental_health_gpt2")

('./mental_health_gpt2/tokenizer_config.json',
 './mental_health_gpt2/special_tokens_map.json',
 './mental_health_gpt2/vocab.json',
 './mental_health_gpt2/merges.txt',
 './mental_health_gpt2/added_tokens.json')

In [38]:
model = GPT2LMHeadModel.from_pretrained("./mental_health_gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("./mental_health_gpt2")

model.config.pad_token_id = tokenizer.pad_token_id

# Testing

In [39]:
# Test

# Question = input("Enter Your Question About Mental_Health : ")
# "Q: Question\nA:"
# Question = "Q: "+Question+"\nA"


Question = "What is depression?"
# apply tokenizer
inputs = tokenizer(Question , padding="max_length",return_tensors="pt",truncation=True,max_length=128 )

# generate output
output_ids = model.generate(inputs["input_ids"],max_new_tokens=50,
                          pad_token_id=tokenizer.pad_token_id,
                         do_sample=True,
                         top_k=10,
                        top_p=0.95, )
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
generated_text

"What is depression?\nA depression is a mental health problem that occurs when your brain or other bodily system fails to make decisions. It is a mental health issue that affects everyone. Your brain is your brain, so it can process information, but it doesn't make decisions"

# combine With RAG

In [40]:
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline

In [41]:
# Load Documents

def load_documents(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    return documents

pdf1 = load_documents("/content/Anxiety-Disorders-Explained-NIMH-1.pdf")
pdf2 = load_documents("/content/NIMH_Depression.pdf")
pdf3 = load_documents("/content/post-traumatic-stress-disorder.pdf")
pdf4 = load_documents("/content/bipolar-disorder_0.pdf")
pdf5 = load_documents("/content/schizophrenia.pdf")

documents = pdf1 + pdf2 + pdf3 + pdf4 + pdf5

In [42]:
# split documents to chunks

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

print(f"Number of chunks {len(texts)}")

Number of chunks 348


In [43]:
# embedding each chunk

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [69]:
# indexing with FAISS

db = FAISS.from_documents(texts, embeddings)
retriver = db.as_retriever(search_kwargs={'k': 2})

In [79]:
# define LLM model

# hf_pipeline = pipeline("text-generation",model = model,tokenizer = tokenizer)
# llm = HuggingFacePipeline(pipeline = hf_pipeline)


# The modification happens when defining the HuggingFace pipeline
hf_pipeline = pipeline(
    "text-generation", model=model,tokenizer=tokenizer,
      max_new_tokens=300
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)

Device set to use cpu


In [84]:
# qa_chain = RetrievalQA( llm = llm,retriever = retriver )

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,                # Pass your LLM here
    chain_type="map_reduce",      # This is the default chain type (stuffs all documents into the prompt)
    retriever=retriver      # Pass your retriever instance here
)

# Final Testing

- Query2

In [82]:
query2 = "Compare the symptoms of a depressive episode in Bipolar Disorder versus the general symptoms of Major Depression. Are the core feelings of sadness, hopelessness, and indifference common to both?"
result2 = qa_chain.run(query2)
print(result2)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Feeling unusually important, talented, 
or powerful
Feeling hopeless or worthless, or 
thinking about death or suicide
Some people with bipolar disorder may have milder symptoms than others. For 
example, hypomanic episodes may make an individual feel very good and 
productive; they may not feel like anything is wrong. However, family and friends 
may notice the mood swings and changes in activity levels as unusual behavior, 
and depressive episodes may follow hypomanic episodes.

4
Depression
National Institute of Mental Health
What are the symptoms of depression?
People with depressive illnesses do not all experience the 
same symptoms. The severity, frequency and duration of 
symptoms will vary depending on the individual and his or 
her particular illness. 
symptoms
Persistent sad, anxious or “empty” feelings
Feelings of

- Query3

In [83]:
query3 = "What co-occurring mental conditions are mentioned for Bipolar Disorder and Schizophrenia? Does the information suggest that having an anxiety disorder might lead to symptoms of psychosis, and if so, how?"
result3 = qa_chain.run(query3)
print(result3)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Conditions That Can Co-Occur With Bipolar Disorder
Many people with bipolar disorder also have other mental disorders or conditions 
such as anxiety disorders, attention-deficit/hyperactivity disorder (ADHD), misuse 
of drugs or alcohol, or eating disorders. Sometimes people who have severe 
manic or depressive episodes also have symptoms of psychosis, such as 
hallucinations or delusions. The psychotic symptoms tend to match the person’s

hallucinations or delusions. The psychotic symptoms tend to match the person’s 
extreme mood. For example, someone having psychotic symptoms during a 
depressive episode may falsely believe they are financially ruined, while 
someone having psychotic symptoms during a manic episode may falsely believe 
they are famous or have special powers.
Looking at symptoms over the course of the illne